## Setup

In [ ]:
import sys
toolpath = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/src'
sys.path.append(toolpath)

%load_ext autoreload
%autoreload 2

from wtfc_utils import etl_utils as etl

import datetime
# import time
import configparser
import json
import requests
from sqlalchemy import create_engine

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import fitbit
import myfitnesspal
from nokia import NokiaAuth, NokiaApi, NokiaCredentials # Withings


In [ ]:
server_dir = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/'
cfg_file = server_dir + 'config/api_params.cfg'
db_dir = server_dir + 'db/'
backups_dir = db_dir + 'backups/'
db_name = 'weightforecaster'
db_ext = '.db'
db_file_name = db_dir + db_name + db_ext

In [ ]:
db_file_name

## Load DB

In [ ]:
# See: https://pandas.pydata.org/pandas-docs/stable/io.html#advanced-sqlalchemy-queries
engine = create_engine('sqlite:///'+db_file_name)

In [ ]:
with engine.connect() as conn, conn.begin():
    db_df = pd.read_sql_table('fitness', conn, index_col='date', parse_dates=['date'])

In [ ]:
db_df.tail(5)

## Initialize API configs

### Nokia / Withings

See the following:
- https://github.com/orcasgit/python-nokia
- https://github.com/orcasgit/python-nokia/blob/master/nokia/__init__.py

In [ ]:
parser = configparser.ConfigParser()
parser.read(cfg_file)
CLIENT_ID = parser.get('withings', 'client_id')
CLIENT_SECRET = parser.get('withings', 'client_secret')
REDIRECT_URI = parser.get('withings', 'redirect_uri')

auth = NokiaAuth(CLIENT_ID, CLIENT_SECRET, callback_uri=REDIRECT_URI)
authorize_url = auth.get_authorize_url()

# open the following in a browser, click allow, it redirects. Copy the 'code' parameter from that url.
print(authorize_url)

In [ ]:
# set this to the alphanumeric string value appearing for the 'code' parameter in the url
code = 'xxx'

In [ ]:
res = requests.post(url = 'https://account.withings.com/oauth2/token', 
              data = {
                  'grant_type':'authorization_code',
                  'client_id':CLIENT_ID,
                  'client_secret':CLIENT_SECRET,
                  'redirect_uri':REDIRECT_URI,
                  'code':code
              })   
token_dict = json.loads(res.content)
token_dict

In [ ]:
etl.persist_nokia_refresh_token(token_dict, cfg_file)

### Fitbit

See the following:

- https://python-fitbit.readthedocs.io/en/latest/

In a terminal, cd to location of python-fitbit repo (clone it from github: https://github.com/orcasgit/python-fitbit).

Then run the following:

```
python gather_keys_oauth2.py <client_key> <client_secret>
```

This script has a callback function for persisting the refresh token. Make sure the path to the api_params.cfg file is properly set (probably good to test it out…)

### MyFitnessPal

See the following:

- https://github.com/coddingtonbear/python-myfitnesspal

In a terminal, run the following command to set up authentication (locally storing your user credentials):

```
myfitnesspal store-password my_username
```

## Test etl scripting

In [ ]:
db_df_updated = db_df.copy()

In [ ]:
etl.get_target_date_endpoints('steps', db_df_updated)

### Fitbit

In [ ]:
db_df_updated = etl.refresh_steps(cfg_file, engine, db_df_updated)

In [ ]:
db_df_updated.tail(28)

### MyFitnessPal

In [ ]:
db_df_updated = etl.refresh_calories(engine, db_df_updated)

In [ ]:
db_df_updated.tail(28)

### Nokia / Withings

In [ ]:
db_df_updated = etl.refresh_weight(cfg_file, engine, db_df_updated)

In [ ]:
db_df_updated[db_df_updated.index > '2019-10-30']

## Impute weight

In [ ]:
db_df_updated = etl.impute_missing_weights(engine, db_df_updated)

In [ ]:
db_df_updated[db_df_updated.index > '2019-10-30']

## Add rolling week-averaged columns

In [ ]:
db_df_updated = etl.add_roll_avg_columns(engine, db_df_updated)

In [ ]:
db_df_updated.tail(21)